In [2]:
import math
import random
import numpy as np #need for speedup the calculations

Option price can then be calculated by:

- generate a large number of approximations for the stock price at maturity
- determine the average pay-off from the stock prices
- take the risk-free interest rate discount to obtain the option price

In [3]:
class EuropeanVanillaPricing:
    def __init__(self, param):
        self.pc = param.pc
        self.S = param.S
        self.K = param.K
        self.T = param.T
        self.r = param.r
        self.sigma = param.sigma
        self.iterations = param.iterations


In [ ]:
    def getPrice(self):
            total = 0.0
            mul = self.S*math.exp(self.T*(self.r - 0.5*self.sigma**2))
            for i in range(0, self.iterations):
                rand = random.gauss(0,1)
                if self.pc == 'call':
                    total += max(mul*math.exp(math.sqrt(self.T*self.sigma**2)*rand) - self.K, 0.0)
                elif self.pc == 'put':
                    total += max(self.K - mul*math.exp(math.sqrt(self.T*self.sigma**2)*rand), 0.0)

            return math.exp(-1.0*self.r*self.T)*(total/self.iterations)

The above function performs poorly because of the for loop used to generate pay-offs, it is possible to use NumPy to implement vector operations for better performances.

Then we construct a $N\times 2$ matrix of zeros where $N$ is the number of iterations and generate a vector containing $N$ normally distributed random numbers with mean zero and unit variance.

In [ ]:
#calc = np.zeros([self.iterations, 2])
#rand = np.random.normal(0, 1, [1, self.iterations])

The second column of the $N\times 2$ matrix is then assigned to $S(T)-K$ or $K-S(T)$ for a call or put option respectively.

In [ ]:
#mult = self.S*np.exp(self.T*(self.r - 0.5*self.sigma*self.sigma))
 
#if self.pc == 'call':
#    calc[:,1] = mult*np.exp(np.sqrt((self.sigma**2)*self.T)*rand) - self.K
#elif self.pc == 'put':
#    calc[:,1] = self.K - mult*np.exp(np.sqrt((self.sigma**2)*self.T)*rand)


We now determine the maximum value in each row of the $N\times 2$ matrix using the NumPy amax command. The average pay-off is then calculated by summing together the returned vector and dividing by the number of iterations. The average pay-off is then multiplied by $e^{-rT}$. Hence, the NumPy optimised pricing function takes the form:



In [5]:
    def getMCPrice(self):
            'Determine the option price using a Monte Carlo approach'
            calc = np.zeros([self.iterations, 2])
            rand = np.random.normal(0, 1, [1, self.iterations])
            mult = self.S*np.exp(self.T*(self.r - 0.5*self.sigma**2))

            if self.pc == 'call':
                calc[:,1] = mult*np.exp(np.sqrt((self.sigma**2)*self.T)*rand) - self.K
            elif self.pc == 'put':
                calc[:,1] = self.K - mult*np.exp(np.sqrt((self.sigma**2)*self.T)*rand)

            avg_po = np.sum(np.amax(calc, axis=1))/float(self.iterations)

            return np.exp(-1.0*self.r*self.T)*avg_po


Include the exact Black-Scholes for European options prices.

In [9]:
    def getBlackScholesPrice(self):
            'Determine the option price using the exact Black-Scholes expression.'
            d1 =  np.log(self.S/self.K) + (self.r + 0.5*self.sigma**2)*self.T
            d1 /= self.sigma*np.sqrt(self.T)

            d2 = d1 - self.sigma*np.sqrt(self.T)

            call = self.S*self.ncdf(d1)
            call -= self.K*np.exp(-1.0*self.r*self.T)*self.ncdf(d2)

            if self.pc == 'call':
                return call
            elif self.pc == 'put':
                return self.applyPCParity(call)
 



In [10]:
    def ncdf(self, x):
            'Cumulative distribution function for the standard normal distribution'
            return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0
 


In [11]:
    def applyPCParity(self, call):
            'Make use of put-call parity to determine put price.'
            return self.K*np.exp(-1.0*self.r*self.T) - self.S + call

Gathering code together for testing with given parameters

In [37]:
import numpy as np
import math
import time
 
class EuropeanVanillaPricing:
    def __init__(self, param):
        self.method = param.method
        self.pc = param.pc
        self.S = param.S
        self.K = param.K
        self.T = param.T
        self.r = param.r
        self.sigma = param.sigma
        if self.method == 'mc':
            self.iterations = param.iterations
 
    def getPrice(self):
        if self.method == 'mc':
            return self.getMCPrice()
        elif self.method == 'exact':
            return self.getBlackScholesPrice()
 
    def getMCPrice(self):
        'Determine the option price using a Monte Carlo approach'
        calc = np.zeros([self.iterations, 2])
        rand = np.random.normal(0, 1, [1, self.iterations])
        mult = self.S*np.exp(self.T*(self.r - 0.5*self.sigma**2))
 
        if self.pc == 'call':
            calc[:,1] = mult*np.exp(np.sqrt((self.sigma**2)*self.T)*rand) - self.K
        elif self.pc == 'put':
            calc[:,1] = self.K - mult*np.exp(np.sqrt((self.sigma**2)*self.T)*rand)
 
        avg_po = np.sum(np.amax(calc, axis=1))/float(self.iterations)
  
        return np.exp(-1.0*self.r*self.T)*avg_po
 
    def getBlackScholesPrice(self):
        'Determine the option price using the exact Black-Scholes expression.'
        d1 =  np.log(self.S/self.K) + (self.r + 0.5*self.sigma**2)*self.T
        d1 /= self.sigma*np.sqrt(self.T)
 
        d2 = d1 - self.sigma*np.sqrt(self.T)
 
        call = self.S*self.ncdf(d1)
        call -= self.K*np.exp(-1.0*self.r*self.T)*self.ncdf(d2)
 
        if self.pc == 'call':
            return call
        elif self.pc == 'put':
            return self.applyPCParity(call)
 
    def ncdf(self, x):
        'Cumulative distribution function for the standard normal distribution'
        return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0
 
    def applyPCParity(self, call):
        'Make use of put-call parity to determine put price.'
        return self.K*np.exp(-1.0*self.r*self.T) - self.S + call


In [63]:
class Parameters:
  pass
 
testParam = Parameters()
testParam.T = 1.
testParam.S = 100.0
testParam.K = 99.0
testParam.sigma = 0.20
testParam.r = 0.01;
testParam.method = 'mc'
testParam.pc = 'call'
testParam.iterations = 500
 
option = EuropeanVanillaPricing(testParam)
print ('Method Monte Carlo')
t0 = time.time()
print ('Price: ' + str(option.getPrice()))
t1 = time.time()
print ('Iterations: ' + str(testParam.iterations))
print( 'Time: ' +str(t1-t0) + 's')

Method Monte Carlo
Price: 8.608798307065253
Iterations: 500
Time: 0.0010030269622802734s


In [64]:
class Parameters:
  pass
 
testParam = Parameters()
testParam.T = 1.
testParam.S = 100.0
testParam.K = 99.0
testParam.sigma = 0.20
testParam.r = 0.01;
testParam.method = 'exact'
testParam.pc = 'call'
testParam.iterations = 500
 
option = EuropeanVanillaPricing(testParam)
print ('Black&Scholes')
t0 = time.time()
print ('Price: ' + str(option.getPrice()))
t1 = time.time()
print ('Iterations: ' + str(testParam.iterations))
print( 'Time: ' +str(t1-t0) + 's')

Black&Scholes
Price: 8.918504421014205
Iterations: 500
Time: 0.0s


In [65]:
class Parameters:
  pass
 
testParam = Parameters()
testParam.T = 1.
testParam.S = 100.0
testParam.K = 99.0
testParam.sigma = 0.20
testParam.r = 0.01;
testParam.method = 'mc'
testParam.pc = 'put'
testParam.iterations = 500
 
option = EuropeanVanillaPricing(testParam)
print ('Method Monte Carlo')
t0 = time.time()
print ('Price: ' + str(option.getPrice()))
t1 = time.time()
print ('Iterations: ' + str(testParam.iterations))
print( 'Time: ' +str(t1-t0) + 's')

Method Monte Carlo
Price: 6.853152981196179
Iterations: 500
Time: 0.0s


In [66]:
class Parameters:
  pass
 
testParam = Parameters()
testParam.T = 1.
testParam.S = 100.0
testParam.K = 99.0
testParam.sigma = 0.20
testParam.r = 0.01;
testParam.method = 'exact'
testParam.pc = 'put'
testParam.iterations = 500
 
option = EuropeanVanillaPricing(testParam)
print ('Black&Scholes')
t0 = time.time()
print ('Price: ' + str(option.getPrice()))
t1 = time.time()
print ('Iterations: ' + str(testParam.iterations))
print( 'Time: ' +str(t1-t0) + 's')

Black&Scholes
Price: 6.933437962181841
Iterations: 500
Time: 0.0s
